In [1]:
import logging
import os
import shutil
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean


/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digen = pickle.load(f)

#print(digen.keys())

digenkeys = list(digen.keys())

#print(digenkeys)
#print(digen['digen8_4426'])
digennoise = {}
traincsv = {}
testcsv = {}
train_best_preformance = {}
test_final_preformance ={}
lossfxn = {}

In [3]:
for i in digenkeys:
    print(i) 
    data = pd.DataFrame(digen[i]['X'])
    yval = pd.DataFrame(digen[i]['y'])
    current_noise = digen[i]['noise']
    yval.columns = ['y']
    yval.rename(columns = {'0':'y'}, inplace = True)
    out = pd.concat([data, yval], axis=1)
    out.index.name = 'InstanceId'
    digennoise[i] = current_noise
    if current_noise == 0.0: 
        outstring = './data/DigenStudy/No_Noise/' + i + '.csv'
        tempstring = './data/testfolder/' + i + '.csv'
        out.to_csv(outstring)
        out.to_csv(tempstring)
    else:
        outstring = './data/DigenStudy/Noisy/' + i + '.csv'
        tempstring = './data/testfolder/' + i + '.csv'
        out.to_csv(outstring)
        out.to_csv(tempstring)
    
    X_train, X_test,y_train, y_test = train_test_split(data, yval,test_size=0.08, shuffle=True)

    train_set = pd.concat([X_train, y_train], axis =1)
    train_set.index.name = 'InstanceId'
    tempstring = './data/testfolder/' + i +'_train.csv'
    train_set.to_csv(tempstring)
    cleanrun = OptimizeClean(dataset_name=[i], n_trials=100, data_path='./data/testfolder')
    cleanrun.run()
    optimal = cleanrun.param
    current_auc = cleanrun.goal

    tempstring = './data/testfolder/' + i +'_train.csv'
    test_set = pd.concat([X_test, y_test], axis =1)
    test_set.index.name = 'InstanceId'

    try:
        most_recent_run = AutoRunner(**optimal, dataset_names=[i],data_path='./data/testfolder',
                                        gen_report=False, clean=False, class_label='target', 
                                        instance_label='InstanceID', ml_algorithms=["NB", "LR", "DT", "EN", "XGB", "LGB", "CGB", "SVM","GB", "RF"], 
                                        exclude=["GB", "RF","ANN","KNN","GP", 'eLCS', 'XCS', "ExSTraCS"]) # "XGB", "LGB", "CGB", "SVM","GB", "RF"
        output_csv = most_recent_run.run(run_para=False)
        performance = pd.read_csv(output_csv)
        summary_chart = performance
        goal = performance["ROC AUC"].max()
        best_model = performance.loc[performance["ROC AUC"].idxmax()][0]
        png_out = output_csv.removesuffix('Summary_performance_mean.csv')
        png_out = png_out + 'Summary_ROC.png'
        final_model_comparison = plt.show(png_out)
    except:
        print('EXCEPTION')
        
    print(current_auc-goal)
        


    



[I 2023-07-06 18:20:37,985] A new study created in memory with name: no-name-6c254f61-7413-4762-bef2-79bfe8dfb577


digen8_4426


  0%|          | 0/100 [00:00<?, ?it/s]INFO: ------------------------------------------------------- 
INFO: Loading Dataset: digen8_4426_train
INFO: Validating and Identifying Feature Types...
INFO: Running Initial EDA:
INFO: Initial Data Counts: ----------------
INFO: Instance Count = 9200
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       4758
1      0       4442
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
5               X0                 X5    -0.025402
68              X6       

EXCEPTION
[I 2023-07-06 18:20:39,923] Trial 0 finished with value: 0.0 and parameters: {'categorical_cutoff': 12, 'featureeng_missingness': 0.15000000000000002, 'cleaning_missingness': 0.3, 'correlation_removal_threshold': 0.9, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Stratified', 'n_splits': 4}. Best is trial 0 with value: 0.0.


INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
5               X0                 X5    -0.025402
68              X6                 X8    -0.022828
56              X5                 X6    -0.020996
23              X2                 X3     0.020983
15              X1                 X5    -0.018678
27              X2                 X7     0.017692
34              X3                 X4     0.016660
59              X5                 X9    -0.015881
18              X1                 X8    -0.015024
28              X2                 X8    -0.014774
INFO: No Features with correlation higher that parameter
INFO: Running Basic Exploratory Analysis...
INFO: Processed Data Counts

In [ ]:
#Noisy checks:



type: 'float' object is not subscriptable